# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [218]:
problem = '''A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared. 
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. '''

## 2. Ask for parameters

In [247]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [248]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

* **X1:**  Number of 1.5L COLI units produced daily. 
* **X2:** Number of 1.5L PEPSA units produced daily.
* **Y:** Hectoliters of syrup purchased and processed daily. 


# 2. Ask for objective

In [249]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [250]:
Markdown(response2.text)

Maximize:  49000 * X1 - X1^2 + 30 * X2 - 2 * X2^2 - 150 * Y 


# 3. Ask for constraints

In [251]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [252]:
Markdown(response3.text)

## Constraints:

1. **Syrup Conversion:** 
   *  X1 <= 40 * Y  (Maximum COLI units from Y hectoliters of syrup)
   *  X2 <= 20 * Y  (Maximum PEPSA units from Y hectoliters of syrup) 

2. **Production Line Capacity:**
   *  X1 + X2 <= 7100 * 8  (Total units produced daily cannot exceed line capacity)

3. **Minimum PEPSA Production:**
   *  X2 >= 0.5 * X1 (At least half the amount of COLI must be PEPSA)

4. **Non-Negativity:**
   *  X1 >= 0 
   *  X2 >= 0
   *  Y >= 0 


## 3. Generate the pyomo code

In [253]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [254]:
#Showing the code in a formatted way
Markdown(response4.text)

```python
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define variables
model.X1 = Var(domain=NonNegativeReals)  # Number of 1.5L COLI units produced daily
model.X2 = Var(domain=NonNegativeReals)  # Number of 1.5L PEPSA units produced daily
model.Y = Var(domain=NonNegativeReals)  # Hectoliters of syrup purchased and processed daily

# Define objective function
def objective_rule(model):
    return 49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2 - 150 * model.Y
model.profit = Objective(rule=objective_rule, sense=maximize)

# Define constraints
model.syrup_coli = Constraint(expr=model.X1 <= 40 * model.Y)
model.syrup_pepsa = Constraint(expr=model.X2 <= 20 * model.Y)
model.production_capacity = Constraint(expr=model.X1 + model.X2 <= 7100 * 8)
model.min_pepsa = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('ipopt')  # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal solution:")
print("X1 (COLI units):", value(model.X1))
print("X2 (PEPSA units):", value(model.X2))
print("Y (Syrup hectoliters):", value(model.Y))
print("Maximum Profit:", value(model.profit))
```


## 4. Input problem data and try running the generated code

In [255]:
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define variables
model.X1 = Var(domain=NonNegativeReals)  # Number of 1.5L COLI units produced daily
model.X2 = Var(domain=NonNegativeReals)  # Number of 1.5L PEPSA units produced daily
model.Y = Var(domain=NonNegativeReals)  # Hectoliters of syrup purchased and processed daily

# Define objective function
def objective_rule(model):
    return 49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2 - 150 * model.Y
model.profit = Objective(rule=objective_rule, sense=maximize)

# Define constraints
model.syrup_coli = Constraint(expr=model.X1 <= 40 * model.Y)
model.syrup_pepsa = Constraint(expr=model.X2 <= 20 * model.Y)
model.production_capacity = Constraint(expr=model.X1 + model.X2 <= 7100 * 8)
model.min_pepsa = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('ipopt')  # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal solution:")
print("X1 (COLI units):", value(model.X1))
print("X2 (PEPSA units):", value(model.X2))
print("Y (Syrup hectoliters):", value(model.Y))
print("Maximum Profit:", value(model.profit))

Status: ok
Termination condition: optimal
Optimal solution:
X1 (COLI units): 16337.083333340031
X2 (PEPSA units): 8168.541666660054
Y (Syrup hectoliters): 408.4270833494343
Maximum Profit: 400350437.76073945


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [256]:
print(response.text)

## Variables:

* **X1:**  Number of 1.5L COLI units produced daily. 
* **X2:** Number of 1.5L PEPSA units produced daily.
* **Y:** Hectoliters of syrup purchased and processed daily. 



In [257]:
print(response2.text)

Maximize:  49000 * X1 - X1^2 + 30 * X2 - 2 * X2^2 - 150 * Y 



In [258]:
print(response3.text)

## Constraints:

1. **Syrup Conversion:** 
   *  X1 <= 40 * Y  (Maximum COLI units from Y hectoliters of syrup)
   *  X2 <= 20 * Y  (Maximum PEPSA units from Y hectoliters of syrup) 

2. **Production Line Capacity:**
   *  X1 + X2 <= 7100 * 8  (Total units produced daily cannot exceed line capacity)

3. **Minimum PEPSA Production:**
   *  X2 >= 0.5 * X1 (At least half the amount of COLI must be PEPSA)

4. **Non-Negativity:**
   *  X1 >= 0 
   *  X2 >= 0
   *  Y >= 0 



In [259]:
print(response4.text)

```python
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define variables
model.X1 = Var(domain=NonNegativeReals)  # Number of 1.5L COLI units produced daily
model.X2 = Var(domain=NonNegativeReals)  # Number of 1.5L PEPSA units produced daily
model.Y = Var(domain=NonNegativeReals)  # Hectoliters of syrup purchased and processed daily

# Define objective function
def objective_rule(model):
    return 49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2 - 150 * model.Y
model.profit = Objective(rule=objective_rule, sense=maximize)

# Define constraints
model.syrup_coli = Constraint(expr=model.X1 <= 40 * model.Y)
model.syrup_pepsa = Constraint(expr=model.X2 <= 20 * model.Y)
model.production_capacity = Constraint(expr=model.X1 + model.X2 <= 7100 * 8)
model.min_pepsa = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('ipopt')  # You can choose a different solver if you prefer
results = solver.solve(model)

# 